## word2vec と doc2vec

単語や文章を分散表現（意味が似たような単語や文章を似たようなベクトルとして表現）を取得します。

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/ml/lec/text/w2v/w2v_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/ml/lec/text/w2v/w2v_nb.ipynb)

### 筆者の環境
筆者のOSはmacOSです。LinuxやUnixのコマンドとはオプションが異なります。

In [1]:
!sw_vers

ProductName:	Mac OS X
ProductVersion:	10.14.6
BuildVersion:	18G6032


In [2]:
!python -V

Python 3.8.5


基本的なライブラリをインポートしそのバージョンを確認しておきます。tensorflowとkerasuのversionも確認します。

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np

import tensorflow as tf
from tensorflow import keras

print('matplotlib version :', matplotlib.__version__)
print('scipy version :', scipy.__version__)
print('numpy version :', np.__version__)
print('tensorflow version : ', tf.__version__)
print('keras version : ', keras.__version__)

matplotlib version : 3.3.2
scipy version : 1.5.2
numpy version : 1.18.5
tensorflow version :  2.3.1
keras version :  2.4.0


### テキストデータの取得

著作権の問題がない青空文庫からすべての作品をダウンロードしてきます。gitがかなり重いので、最新の履歴だけを取得します。

```bash
git clone --depth 1 https://github.com/aozorabunko/aozorabunko.git
```

実際のファイルはcardsにzip形式として保存されているようです。ディレクトリの個数を確認してみます。

In [4]:
!ls ./aozorabunko/cards/* | wc -l

   19636


zipファイルだけzipsに移動させます。

```bash
find ./aozorabunko/cards/ -name *.zip | xargs -I{} cp {} -t ./zips/
```

In [5]:
!ls ./zips/ | head -n 5

1000_ruby_2956.zip
1001_ruby_2229.zip
1002_ruby_20989.zip
1003_ruby_2008.zip
1004_ruby_2053.zip


In [6]:
!ls ./zips/ | wc -l

   16444


となり、16444個のzipファイルがある事が分かります。こちらをすべて解凍し、ディレクトリを移動させます。

```bash
for i in `ls`; do [[ ${i##*.} == zip ]] && unzip -o $i -d ../texts/; done
```

これで、textｓというディレクトリにすべての作品のテキストファイルがインストールされました。

In [7]:
!ls ./texts/ | grep miyazawa

miyazawa_kenji_zenshu.txt
miyazawa_kenji_zenshuno_kankoni_saishite.txt
miyazawa_kenjino_sekai.txt
miyazawa_kenjino_shi.txt


In [8]:
!ls ./texts/ | grep ginga_tetsudo

ginga_tetsudono_yoru.txt


となり、宮沢賢治関連の作品も含まれていることが分かります。銀河鉄道の夜もあります。

## 銀河鉄道の夜を使ったword2vec

今回はすべてのテキストファイルを対象にするには時間がかかるので、同じ岩手県出身の、高校の先輩でもある宮沢賢治の作品を例に取りword2vecを試してみます。
しかし、ファイルの中身を見てみると、

In [9]:
!head ./texts/ginga_tetsudono_yoru.txt

In [10]:
!nkf --guess ./texts/ginga_tetsudono_yoru.txt

Shift_JIS (CRLF)


となりshift_jisで保存されていることが分かります。

In [11]:
!nkf -w ./texts/ginga_tetsudono_yoru.txt > ginga.txt

と、ディレクトリを変更し、ファイル名も変更します。

In [12]:
!cat ginga.txt | head -n 25


























cat: stdout: Broken pipe


In [13]:
!cat ginga.txt | tail -n 25

となり、ファイルの先頭と、末尾に参考情報が載っているほかは、ちゃんとテキストとしてデータが取れている模様です。
先ず、この辺の前処理を行います。

In [14]:
import re

with open('ginga.txt', mode='r') as f:
  all_sentence = f.read()

全角、半角の空白、改行コード、縦線(|)をすべて削除します。正規表現を利用します。

In [15]:
all_sentence = all_sentence.replace(" ", "").replace("　","").replace("\n","").replace("|","")

《》で囲まれたルビの部分を削除します。正規表現を利用します。

In [16]:
all_sentence = re.sub("《[^》]+》", "", all_sentence)

----------の部分で分割を行い、2番目の要素を取得します。

In [17]:
all_sentence = re.split("\-{8,}", all_sentence)[2]

。で分割し、文ごとにリストに格納します。

In [18]:
sentence_list = all_sentence.split("。")
sentence_list = [ s + "。" for s in sentence_list]
sentence_list[:5]

['［＃７字下げ］一、午后の授業［＃「一、午后の授業」は中見出し］「ではみなさんは、さういふふうに川だと云はれたり、乳の流れたあとだと云はれたりしてゐたこのぼんやりと白いものがほんたうは何かご承知ですか。',
 '」先生は、黒板に吊した大きな黒い星座の図の、上から下へ白くけぶった銀河帯のやうなところを指しながら、みんなに問をかけました。',
 'カムパネルラが手をあげました。',
 'それから四五人手をあげました。',
 'ジョバンニも手をあげやうとして、急いでそのまゝやめました。']

最初の文は不要なので削除します。

In [19]:
sentence_list = sentence_list[1:]
sentence_list[:5]

['」先生は、黒板に吊した大きな黒い星座の図の、上から下へ白くけぶった銀河帯のやうなところを指しながら、みんなに問をかけました。',
 'カムパネルラが手をあげました。',
 'それから四五人手をあげました。',
 'ジョバンニも手をあげやうとして、急いでそのまゝやめました。',
 'たしかにあれがみんな星だと、いつか雑誌で読んだのでしたが、このごろはジョバンニはまるで毎日教室でもねむく、本を読むひまも読む本もないので、なんだかどんなこともよくわからないといふ気持ちがするのでした。']

となり、不要な部分を削除し、一文ごとにリストに格納できました。前処理は終了です。

## janomeによる形態素解析

janomeは日本語の文章を形態素ごとに分解する事が出来るツールです。同じようなツールとして、MecabやGinzaなどがあります。一長一短があると思いますが、ここではjanomeを利用します。

In [65]:
from janome.tokenizer import Tokenizer

t = Tokenizer()

word_list = []
# word_per_sentence_list = []
# for sentence in sentence_list:
#   word_list.extend(list(t.tokenize(sentence, wakati=True)))
#   word_per_sentence_list.append(list(t.tokenize(sentence, wakati=True)))

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
  tokens = t.tokenize(text)
  return [token.base_form for token in tokens if token.part_of_speech.split(',')[0] in['名詞', '動詞']]
    

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
# sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
# word_list = [extract_words(sentence) for sentence in sentence_list] 
for sentence in sentence_list:
  word_list.extend(extract_words(sentence))
print(word_list[:10])
# print(word_per_sentence_list[:5])

['先生', '黒板', '吊す', '星座', '図', '上', '下', 'けぶる', '銀河', '帯']


## 単語のカウント

単語のカウントを行い、出現頻度の高いベスト10を抽出してみます。名詞のみに限定した方が良かったかもしれません。

In [56]:
import collections

count = collections.Counter(word_list)
count.most_common()[:10]
dict(count.most_common())['銀河']
dict(count.most_common())['ジョバンニ']

191

## gensimに含まれるword2vecを用いた学習

word2vecを用いて、word_listの分散表現を取得します。使い方はいくらでも検索できますので、ここでは割愛します。単語のリストを渡せば、ほぼ自動的に分散表現を作ってくれます。

In [84]:
from gensim.models import word2vec

model = word2vec.Word2Vec(word_list, size=100, min_count=5, window=5, iter=1000, sg=0)

['先生',
 '黒板',
 '吊す',
 '星座',
 '図',
 '上',
 '下',
 'けぶる',
 '銀河',
 '帯',
 'やう',
 'ところ',
 '指す',
 'みんな',
 '問',
 'かける',
 'カムパネルラ',
 '手',
 'あげる',
 '四',
 '五',
 '人',
 '手',
 'あげる',
 'ジョバンニ',
 '手',
 'あげる',
 'やう',
 '急ぐ',
 'やめる',
 'あれ',
 'みんな',
 '星',
 'いつか',
 '雑誌',
 '読む',
 'の',
 'このごろ',
 'ジョバンニ',
 '毎日',
 '教室',
 '本',
 '読む',
 'ひま',
 '読む',
 '本',
 'こと',
 'わかる',
 '気持ち',
 'する',
 'の',
 '先生',
 'それ',
 '見る',
 '附ける',
 'の',
 'ジョバンニ',
 'さん',
 'あなた',
 'わかる',
 'ゐる',
 'する',
 'ジョバンニ',
 '勢',
 '立ちあがる',
 '立つ',
 '見る',
 'それ',
 '答',
 'へる',
 'こと',
 'できる',
 'の',
 'ザネリ',
 '前',
 '席',
 'ふり',
 'ジョバンニ',
 '見る',
 'く',
 'わら',
 'ひる',
 'ジョバンニ',
 'どぎまぎ',
 'する',
 '赤',
 'なる',
 'しまふ',
 '先生',
 '云',
 'ひる',
 '望遠鏡',
 '銀河',
 'よる',
 'くる',
 '調べる',
 '銀河',
 '大体',
 'する',
 '星',
 'ジョバンニ',
 '思ふ',
 'こんど',
 '答',
 'へる',
 'こと',
 'できる',
 '先生',
 '困る',
 'やう',
 'する',
 '眼',
 'カムパネルラ',
 '方',
 '向ける',
 'カムパネルラ',
 'さん',
 '名指す',
 '元気',
 '手',
 'あげる',
 'カムパネルラ',
 'ぢ',
 'ぢ',
 '立ち上る',
 '答',
 'できる',
 '先生',
 '意外',
 'やう',
 'ぢ',
 'カムパネルラ',
 '見る',
 'ゐる',
 '急ぐ',
 '云',
 'ひる',


### 分散行列

In [75]:
model.wv.vectors

array([[ 0.6690906 , -1.8134489 , -0.50446075, ...,  0.22950223,
        -0.24067923, -0.45016605],
       [-0.0081544 ,  0.88565207, -0.6879916 , ...,  0.37250426,
        -0.37231675, -0.23907655],
       [-0.06978781, -0.4953329 , -0.1721944 , ..., -0.34273872,
        -0.676891  , -0.7721713 ],
       ...,
       [ 0.09245484, -0.6152532 , -0.20881364, ...,  0.04918382,
         0.10831165,  0.15404673],
       [ 0.6117021 , -0.9071201 ,  0.8482464 , ...,  0.27837202,
         0.4135082 ,  0.03481499],
       [-3.1874008 , -0.96890706,  1.3699456 , ..., -1.5262604 ,
        -0.79284537, -0.08142332]], dtype=float32)

### 分散行列の形状確認

443個の単語について、100次元のベクトルが生成されました。

In [76]:
model.wv.vectors.shape

(408, 100)

全単語数は、

In [77]:
len(set(word_list))

2019

ですが、word2vecのmin_countを5にしているので、その文単語数が少なくなっています。

In [83]:
model.wv.index2word[:10]
print(model.__dict__['wv']['銀河'])

KeyError: "word '銀河' not in vocabulary"

In [79]:
model.wv.vectors[0]

array([ 0.6690906 , -1.8134489 , -0.50446075,  1.0823753 ,  1.0012556 ,
        0.17782725,  0.39993393, -0.7516008 ,  1.2903652 , -0.79781705,
        0.54625   ,  0.18831149, -0.10043006, -0.68015933, -0.22242035,
        0.72337776, -0.31982303, -0.9627689 , -0.22933453, -0.04989067,
        0.16735213, -0.02974823, -1.3249292 , -0.27127397,  0.42874482,
        0.01675199,  0.8601299 , -0.85613954, -0.79393893,  0.12290027,
        0.6677782 ,  0.4430345 , -0.15914361,  0.92404836,  0.7163351 ,
        0.27910623, -0.09720881,  0.68278235,  1.1329095 , -0.7275171 ,
       -0.99282736,  0.09739671,  1.4512872 , -0.29004535,  1.0013556 ,
       -0.78484267, -0.44537067, -0.17693432,  0.00596993, -0.2871559 ,
       -1.0671868 ,  0.35299167,  0.6387847 , -1.3476065 , -0.51196575,
       -0.09386528,  0.45643848,  0.6014701 , -0.29185364, -0.6555386 ,
        0.3910473 , -0.324209  , -0.5417036 ,  0.08710421, -1.1519334 ,
        0.08187845,  0.7924016 , -0.00519154, -0.2600619 ,  0.96

In [82]:
model.wv.__getitem__("銀河")

KeyError: "word '銀河' not in vocabulary"

### cos類似度による単語抽出

ベクトルの内積を計算することにより、指定した単語に類似した単語をその$\cos$の値と一緒に抽出する事ができます。

In [81]:
print(model.wv.most_similar("銀河"))
print(model.wv.most_similar("本"))
print(model.wv.most_similar("ジョバンニ"))

KeyError: "word '銀河' not in vocabulary"

### 単語ベクトルによる演算

足し算するにはpositiveメソッドを引き算にはnegativeメソッドを利用します。

まず、銀河＋男を計算します。

In [ ]:
model.wv.most_similar(positive=["銀河", "ジョバンニ"])

次に銀河＋ジョバンニー家を計算します。

In [ ]:
model.wv.most_similar(positive=["銀河", "ジョバンニ"], negative=["家"])

## doc2vec

文章毎にタグ付けされたTaggedDocumentを作成します。

In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

tagged_doc_list = []

for i, sentence in enumerate(word_per_sentence_list):
  tagged_doc_list.append(TaggedDocument(sentence, [i]))

print(tagged_doc_list[0])

In [ ]:
model = Doc2Vec(documents=tagged_doc_list, vector_size=100, min_count=5, window=5, epochs=20, dm=0)

In [ ]:
word_per_sentence_list[0]

In [ ]:
model.docvecs[0]

most_similarで類似度が高い文章のIDと類似度を取得することが出来ます。

In [ ]:
model.docvecs.most_similar(0)

In [ ]:
for p in model.docvecs.most_similar(0):
  print(word_per_sentence_list[p[0]])

感覚的ですが、似たような文章が抽出されています。